In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# SQLite

In [ ]:
!apt install sqlite3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  sqlite3-doc
The following NEW packages will be installed:
  sqlite3
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 752 kB of archives.
After this operation, 2,482 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 sqlite3 amd64 3.22.0-1ubuntu0.4 [752 kB]
Fetched 752 kB in 1s (980 kB/s)
Selecting previously unselected package sqlite3.
(Reading database ... 155455 files and directories currently installed.)
Preparing to unpack .../sqlite3_3.22.0-1ubuntu0.4_amd64.deb ...
Unpacking sqlite3 (3.22.0-1ubuntu0.4) ...
Setting up sqlite3 (3.22.0-1ubuntu0.4) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


# 폴더(path) 생성

In [ ]:
# 기본 데이터베이스에 해당하는 파일 디렉토리(모델 넣기)
!mkdir datafile

mkdir: cannot create directory ‘datafile’: File exists


- datafile 에 학습모델, 분류파일 넣기

In [ ]:
# 템플릿 디렉토리 생성
!mkdir templates

mkdir: cannot create directory ‘templates’: File exists


In [ ]:
# static 디렉토리 생성
!mkdir static

In [ ]:
# img 디렉토리 생성
!mkdir img

- templates 디렉토리에 main.html / result.html 파일 넣기
- img 디렉토리에 사용image(png/svg등) 넣기
- static 디렉토리에 style.css 파일 넣기

In [ ]:
file_path = './datafile/' # 데이터베이스 path

# 기본모듈(pd/np) 임포트

In [ ]:
import numpy as np
import pandas as pd

# 모델 =====
- 모델 불러오기

In [ ]:
# 모델 임포트


In [ ]:
# 모델 로드


# 이미지 모듈 =====

In [ ]:
# 추가 모듈 임포트
import json
from PIL import Image

In [ ]:
img_path = './image/'

# 이미지 Predict =====

In [ ]:
# 추출된 인덱스 값 1
# 인덱스의 라벨 값 2

# 데이터 =====
- 영양정보 DB 가져오기

In [ ]:
DATA = pd.read_excel('/content/drive/MyDrive/영양4조/dataset/음식 이미지 및 영양정보 텍스트/nutritionDB/음식분류 AI 데이터 영양DB.xlsx', index_col='음 식 명')
DATA

,중량(g),에너지(kcal),탄수화물(g),당류(g),지방(g),단백질(g),칼슘(mg),인(mg),나트륨(mg),칼륨(mg),마그네슘(mg),철(mg),아연(mg),콜레스테롤(mg),트랜스지방(g)
음 식 명,,,,,,,,,,,,,,,
쌀밥,210.0,334.8000,73.71000,0.000000,0.45000,5.76000,3.6,126,59.40000,146.7,-,0.36,1.35,0.000,0
기타잡곡밥,200.0,302.3620,65.52110,0.000000,0.74620,6.70920,12.06,117.836,3.39200,166.718,6.9,1.7447,1.8264,0.000,0
콩밥,200.0,322.9000,65.84600,0.000000,1.66600,8.42800,24.7,126.9,4.10000,258.4,22.5,1.74,1.61,0.000,0
보리밥,200.0,316.1000,70.57000,0.000000,0.14000,5.55000,6.7,68.8,4.50000,100.6,0,1.26,1.41,0.000,0
돌솥밥,350.0,528.8550,101.85000,0.000000,8.34700,10.18700,61.455,193.72,618.31500,624.32,57.785,3.321,2.582,2.365,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
다식,30.0,105.1665,20.81070,6.455700,1.72860,3.54630,1.4274,100.0665,3.84900,206.7144,-,0.5175,0.0411,0.000,0
약과,30.0,113.8494,22.18320,2.151900,1.23870,2.57400,4.449,28.23,16.45980,59.4219,-,0.5661,0.1431,0.000,0
유과,30.0,129.0600,24.13650,1.120950,3.50400,0.36090,1.2084,3.6,7.05540,0.0285,-,0.24,0.1968,0.000,0


# 영양성분정보 추출 함수 =====

In [ ]:
# 음식명별 영양성분 정보
Nutrition = {'중량': ['중량(g)'],
             '성분': ['에너지(kcal)', '콜레스테롤(mg)', '트랜스지방(g)', '탄수화물(g)', '당류(g)', '지방(g)',
                    '단백질(g)', '칼슘(mg)', '인(mg)', '나트륨(mg)', '칼륨(mg)', '마그네슘(mg)', '철(mg)', '아연(mg)']}

In [ ]:
names = ['곤드레밥', '라면']

In [ ]:
# predict된 음식명에 따라 영양성분 출력 함수 정의
def nutrition_info(names):
    # 예측된 음식명 담기
    food_name = []
    # 없는 음식명 담기
    food_name_x = []
    # 예측된 음식의 영양성분 담기
    food_info = {}
    # 예측된 음식명들중에 하나씩 반복해서 이름은 음식명에 담고, 영양정보는 영양성분에 담기
    for name in names:
        if name in DATA.index:
            food_name.append(name)
            food_info[name] = dict(DATA.loc[name][Nutrition['성분']])
        # 음식명이 없으면 없는 음식명에 담고 없는 정보를 출력할 준비 함
        else:
            food_name_x.append(name)
            a = '없는 정보 :', food_name_x
    # 결과1: 없는음식명이 없다면 예측된 음식명 리스트와 그에대한 영양성분 정보 출력
    if food_name_x == []:
        return food_name, food_info
    # 결과2: 음식명이 없으면 없는음식명 정보를 출력
    elif food_name == []:
        return  print(a)
    # 결과3: 음식명이 있고, 없는것도 있으면 없는 음식명정보도 출력하고 예측된 음식명 리스트와 영양성분정보 출력
    else:
        print(a)
        return food_name, food_info

In [ ]:
nutrition_info(names)

(['곤드레밥', '라면'],
 {'곤드레밥': {'나트륨(mg)': 88.753,
   '단백질(g)': 14.14,
   '당류(g)': 0.0,
   '마그네슘(mg)': 22.400000000000002,
   '아연(mg)': 1.7779999999999998,
   '에너지(kcal)': 506.8,
   '인(mg)': 95.47999999999999,
   '지방(g)': 8.372,
   '철(mg)': 1.8760000000000001,
   '칼륨(mg)': 815.9235000000001,
   '칼슘(mg)': 30.324,
   '콜레스테롤(mg)': 0.0,
   '탄수화물(g)': 108.05199999999999,
   '트랜스지방(g)': 0},
  '라면': {'나트륨(mg)': 1695.5009999999997,
   '단백질(g)': 14.077000000000002,
   '당류(g)': 0.0,
   '마그네슘(mg)': 0.97,
   '아연(mg)': 1.1909999999999998,
   '에너지(kcal)': 509.32800000000003,
   '인(mg)': 145.281,
   '지방(g)': 17.748,
   '철(mg)': 1.226,
   '칼륨(mg)': 374.069,
   '칼슘(mg)': 32.993,
   '콜레스테롤(mg)': 2.344,
   '탄수화물(g)': 83.16800000000002,
   '트랜스지방(g)': 0}})

In [ ]:
# 음식별 영양성분 더하기
def nutri_add(names):
    # 전체음식의 영양성분을 한번에 담을 변수 생성 Nutri
    Nutri = {'에너지(kcal)': 0, '콜레스테롤(mg)': 0, '트랜스지방(g)': 0, '탄수화물(g)': 0, '당류(g)': 0, '지방(g)': 0,
    '단백질(g)': 0, '칼슘(mg)': 0, '인(mg)': 0, '나트륨(mg)': 0, '칼륨(mg)': 0, '마그네슘(mg)': 0, '철(mg)': 0, '아연(mg)': 0}
    # 추출된 전체 음식의 성분별로 영양성분 더하여 dict에 저장
    for name in names[0]:
        for k in Nutrition['성분']:
            Nutri[k] += names[1][name][k]
    return Nutri

In [ ]:
result = nutri_add(nutrition_info(names))
result

{'나트륨(mg)': 1784.2539999999997,
 '단백질(g)': 28.217000000000002,
 '당류(g)': 0.0,
 '마그네슘(mg)': 23.37,
 '아연(mg)': 2.9689999999999994,
 '에너지(kcal)': 1016.128,
 '인(mg)': 240.761,
 '지방(g)': 26.12,
 '철(mg)': 3.1020000000000003,
 '칼륨(mg)': 1189.9925,
 '칼슘(mg)': 63.31700000000001,
 '콜레스테롤(mg)': 2.344,
 '탄수화물(g)': 191.22000000000003,
 '트랜스지방(g)': 0}

# 가공정보 추출 함수 =====

In [ ]:
DATA2 = pd.read_excel('/content/drive/MyDrive/영양4조/남진영/[최종]영양권장량.xlsx')
DATA2

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,성별,연령,영양성분,권장(g/일),상한(g/일)
0,남자,19-29,탄수화물(g/일),130,-
1,남자,30-49,탄수화물(g/일),130,-
2,남자,50-64,탄수화물(g/일),130,-
3,여자,19-29,탄수화물(g/일),130,-
4,여자,30-49,탄수화물(g/일),130,-
...,...,...,...,...,...
61,남자,30-49,아연(mg/일),10,35
62,남자,50-64,아연(mg/일),10,35
63,여자,19-29,아연(mg/일),8,35
64,여자,30-49,아연(mg/일),8,35


In [ ]:
DATA2[(DATA2['성별'] =='남자') & (DATA2['연령']=='19-29') & (DATA2['영양성분']=='탄수화물(g/일)')]

,성별,연령,영양성분,권장(g/일),상한(g/일)
0,남자,19-29,탄수화물(g/일),130,-


In [ ]:
a = DATA2[(DATA2['성별'] == '남자') & (DATA2['연령']=='19-29')]
a[['영양성분','권장(g/일)']]

,영양성분,권장(g/일)
0,탄수화물(g/일),130
6,당류,63.414634
12,지방(g/일),55.026455
18,단백질(g/일),65.0
24,칼슘(mg/일),800
30,인(mg/일),700
36,나트륨(mg/일),"1,500"
42,칼륨(mg/일),"3,500"
48,마그네슘(mg/일),360
54,철(mg/일),10


In [ ]:
gender = '남자'
ages = 28
augemnts = '탄수화물(g/일)'

In [ ]:
if gender == '남자':
    if ages < 19:
        print('마니먹어..')
    elif ages < 30:
        a = DATA2[(DATA2['성별'] == '남자') & (DATA2['연령']=='19-29')]
        for aug in Nutrition['성분']:
            if result[aug].value() >= a[a['영양성분']==aug]['권장(g/일)'].value():
                print(aug, '경고')
        print('남자 20대')
    elif ages < 40:
        DATA2[(DATA2['성별'] == '남자') & (DATA2['연령']=='30-49')]
        print('남자 30대')
    elif ages < 50:
        DATA2[(DATA2['성별'] == '남자') & (DATA2['연령']=='30-49')]
        print('남자 40대')
    elif ages < 60:
        DATA2[(DATA2['성별'] == '남자') & (DATA2['연령']=='50-64')]
        print('남자 50대')
    elif ages < 65:
        DATA2[(DATA2['성별'] == '남자') & (DATA2['연령']=='50-64')]
        print('남자 60~64')
    else:
        print('해당 정보가 없습니다')
elif gender == '여자':
    if ages < 19:
        print('마니먹어..')
    elif ages < 30:
        DATA2[(DATA2['성별'] == '여자') & (DATA2['연령']=='19-29')]
        print('여자 20대')
    elif ages < 40:
        DATA2[(DATA2['성별'] == '여자') & (DATA2['연령']=='30-49')]
        print('여자 30대')
    elif ages < 50:
        DATA2[(DATA2['성별'] == '여자') & (DATA2['연령']=='30-49')]
        print('여자 40대')
    elif ages < 60:
        DATA2[(DATA2['성별'] == '여자') & (DATA2['연령']=='50-64')]
        print('여자 50대')
    elif ages < 65:
        DATA2[(DATA2['성별'] == '여자') & (DATA2['연령']=='50-64')]
        print('여자 60~64')
    else:
        print('해당 정보가 없습니다')

AttributeError: ignored

In [ ]:
# 입력받은 개인정보 기준 영양성분 제한 해주는 부분 함수
def warning(gender, ages, augments):
    a

# 권장섭취량 판별 함수 =====

In [ ]:
# 권장섭취량 DB
DATA2 = pd.read_excel(file_path+'=====.xlsx', index_col='=====')

In [ ]:
# 추출된 값이 권장섭취량 이상인 경우 경고하는 함수 정의
def warning(name):
    warn_ment = str('경고')
    # a
    try:
        a
        result
    except:
        pass  

    # 예측값에서 가져온 영양성분(총합)이 권장섭취량보다 높은지 확인
    if a >= db:
      return result
    else:
      return result

# FASTAPI 설정 =====
- install fastapi, uvicorn 등
- import fastapi, ngrok 등

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn aiofiles python-multipart

In [ ]:
# 모듈 임포트
from fastapi import FastAPI, Request, Form, File, UploadFile
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
import nest_asyncio
from pyngrok import ngrok

import uvicorn

# HTML파일작성
- API 화면 출력용 html 작성

In [ ]:
%%writefile templates/main.html
<!DOCTYPE html>
<html>
	<head>
        <link rel="preconnect" href="https://fonts.googleapis.com" />
		<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin />
		<link
			href="https://fonts.googleapis.com/css2?family=Ubuntu&display=swap"
			rel="stylesheet"
		/>
        
        <!-- Required meta tags -->
		<meta charset="utf-8" />
		<meta name="viewport" content="width=device-width, initial-scale=1" />

        <!-- Bootstrap CSS -->
		<link
        href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css"
        rel="stylesheet"
        integrity="sha384-1BmE4kWBq78iYhFldvKuhfTAU6auU8tT94WrHftjDbrCEXSU1oBoqyl2QvZ6jIW3"
        crossorigin="anonymous"
        />

        <title>Nutrition_4</title>
        <link rel="stylesheet" href="/static/style.css" />
    </head>
	<body>
        <form method="post" action="/print" enctype="multipart/form-data">
            <section class="section">
                <h2 class="title">개인 맞춤형 식단 필터링 서비스</h2>
                <h5 class="subtitle">정보를 입력해주세요</h5>
            <center>
            <div class= 'pt-1 Select Gender'>
                <select name = 'gender'>
                <option type='text', value='male'>남자</option>
                <option type='text', value='female'>여자</option>
                </select>
            </div>
            </center>
            <center>
            <div class= 'pt-1 Input Ages'>
                <input type='number' min='19' max="64" name="ages"></option>
            </div>
            </center>
            <center>
            <div class= 'pt-1 Select Illness'>
                <select name = 'illness'>
                <option type='text', value='A'>비만</option>
                <option type='text', value='B'>당뇨</option>
                <option type='text', value='C'>고혈압</option>
                <option type='text', value='D'>고지혈증</option>
                <option type='text', value='E'>신부전증</option>
                </select>
            </div>
            </center>
            <center>
                <div class='pt-2 mb-2 submitbutton'>
                  <input type="submit", Value=작동>
                </div>
            </center>
            <div class="container file-upload">
                <div class="image-upload-wrap" style="width: 65%; margin: 0 auto;">
                    <input
                        class="file-upload-input"
                        type="file"
                        name='img'
                        onchange="readURL(this);"
                        accept="image/*"
                    />
                    <div class="drag-text">
                        <img src="img/upload.png" class="mt-5 pt-2 upload" style="width: 16%;" />
                        <h3 class="mb-5 pd-5 pt-4 upload-text">음식사진을 올려주세요</h3>
                    </div>
                </div>
                <div class="file-upload-content">
                    <img class="file-upload-image" id="food-image" src="#" alt="your image" />
                    <!-- <p class="result-message"></p> -->
                    <div id="label-container"></div>
                    <div class="image-title-wrap">
                        <button type="button" onclick="removeUpload()" class="remove-image">
                            Remove
                            <span class="image-title">Uploaded Image</span>
                        </button>
                    </div>
                </div>
            </div>
    </body>
	
</html>

- css 파일 작성

In [ ]:
%%writefile static/style.css
body {
	font-family: 'Ubuntu', sans-serif;
	background-color: #fff;
}
.file-upload {
	/* width: 914px; */
	border-radius: 10px;
	border: solid 1.5px #f6f7fa;
	background-color: #f6f7fa;
	padding: 13% 5%;
	margin: 0 auto
	/* width: 600px; */
	
}
.file-upload-btn {
	width: 100%;
	margin: 0;
	color: #fff;
	background: #a61fb2;
	border: none;
	padding: 10px;
	border-radius: 4px;
	border-bottom: 4px solid #a61fb2;
	transition: all 0.2s ease;
	outline: none;
	text-transform: uppercase;
	font-weight: 700;
}
.file-upload-btn:hover {
	background: #1a1ca0;
	color: #fff;
	transition: all 0.2s ease;
	cursor: pointer;
}
.file-upload-btn:active {
	border: 0;
	transition: all 0.2s ease;
}
.file-upload-content {
	display: none;
	text-align: center;
}
.file-upload-input {
	position: absolute;
	margin: 0;
	padding: 0;
	width: 100%;
	height: 100%;
	outline: none;
	opacity: 0;
	cursor: pointer;
}
.image-upload-wrap {
	border: 4px dashed #f582ff;
	position: relative;
}
.image-upload-wrap:hover {
	background-color: #f9bbff;
	border: 4px dashed #fff;
}
.image-dropping {
	background-color: #f9bbff;
	border: 4px dashed #fff;
}
.image-title-wrap {
	padding: 0 15px 15px;
	color: #222;
}
.drag-text {
	text-align: center;
}
.drag-text h3 {
	width: 50%;
	font-weight: 100;
	text-transform: uppercase;
	color: #15824b;
	font-family: 'Ubuntu', sans-serif;
	font-size: 1.5rem;
	line-height: 1.53;
	text-align: center;
	color: #35465d;
	margin: 0 auto;
}
.file-upload-image {
	max-height: 200px;
	max-width: 200px;
	margin: auto;
	padding: 20px;
}
.remove-image {
	width: 200px;
	margin: 0;
	color: #fff;
	background: #cd4535;
	border: none;
	padding: 10px;
	border-radius: 4px;
	border-bottom: 4px solid #b02818;
	transition: all 0.2s ease;
	outline: none;
	text-transform: uppercase;
	font-weight: 700;
}
.remove-image:hover {
	background: #c13b2a;
	color: #fff;
	transition: all 0.2s ease;
	cursor: pointer;
}
.remove-image:active {
	border: 0;
	transition: all 0.2s ease;
}

.nav-distance {
	padding: 2.1% 7.7%;
}

.navbar-light .navbar-brand {
	font-family: 'Ubuntu', sans-serif;
	line-height: 1.68;
	text-align: left;
	color: #35465d;
}

.navbar-light .navbar-toggler {
	border-color: #fff;
}

.section {
	margin-top: 100px;
}

.title {
	font-family: 'Ubuntu', sans-serif;
	line-height: 1.67;
	text-align: center;
	color: #35465d;
}

.subtitle {
	font-family: 'Ubuntu', sans-serif;
	line-height: 1.53;
	text-align: center;
	color: #35465d;
}

/* On screens that are 600px or less, set the background color to olive */
@media screen and (max-width: 600px) {
  html {
    font-size: 10px;
  }
	.navbar-brand {
		font-size: 2rem;
	}
}

.github-link {
	font-family: 'Ubuntu', sans-serif;
	font-weight: bolder;
	font-size: 1.0rem;
	line-height: 1.71;
	text-align: center;
	color: #000000;
	text-decoration: underline;
}

.github-link: hover {
	font-family: 'Ubuntu', sans-serif;
	font-weight: bolder;
	font-size: 1.0rem;
	line-height: 1.71;
	text-align: center;
	color: #2fb004;
	text-decoration: underline;
}

.INFOODD-icon {
	width: 10%
}




.image-upload-wrap {
	object-fit: contain;
	border-radius: 10px;
	border: dashed 1.5px
		#35465d;
	background-color:
		#ffffff;
}

.image-upload-wrap hover {
	object-fit: contain;
	border-radius: 10px;
	border: dashed 1.5px
		#35465d;
	background-color:
		#35465d;
}

.upload-text {
	width: 50%;
	font-family: 'Ubuntu', sans-serif;
	font-size: 1.6 rem;
	line-height: 1.53;
	text-align: center;
	color: #35465d;
	margin: 0 auto;
}

# FastAPI 실행
- 연결 가능

In [ ]:
# FastAPI - 변수설정 및 수정 필요
from fastapi.staticfiles import StaticFiles

app=FastAPI()
templates = Jinja2Templates(directory="templates")
app.mount('/static', StaticFiles(directory='static'), name='static')
app.mount('/img', StaticFiles(directory='img'), name='img')

@app.get('/', response_class=HTMLResponse)
async def read_page(request: Request):
  return templates.TemplateResponse("main.html", {"request": request})

@app.post('/topic', response_class=HTMLResponse)
async def get_page(request: Request, img: UploadFile = File(...), gender: str = Form(...), ages: int = Form(...), illness: str = Form(...)):
  return templates.TemplateResponse("print.html", {"request": request})           

In [46]:
#ngrok authtoken
!ngrok authtoken ===

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print ('Public URL:', ngrok_tunnel.public_url) 
nest_asyncio.apply()
uvicorn.run(app, host='0.0.0.0', port=8000)

Public URL: http://24c0-34-91-29-67.ngrok.io


INFO:     Started server process [493]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [493]
